# Ejercicios Prácticos: Análisis Sintáctico
## Procesadores del Lenguaje - Nivel Universitario Introductorio

Este cuaderno contiene ejercicios prácticos sobre análisis sintáctico.


---
## Ejercicio 1: Identificación de Elementos de una Gramática

Gramática para expresiones booleanas:
```
E -> E or T | T
T -> T and F | F
F -> not F | true | false | ( E )
```

**Tarea:** Identifica terminales, no-terminales y símbolo inicial.

In [1]:
# SOLUCIÓN:
terminales = ['or', 'and', 'not', 'true', 'false', '(', ')']
no_terminales = ['E', 'T', 'F']
simbolo_inicial = 'E'

print(f"Terminales: {terminales}")
print(f"No-terminales: {no_terminales}")
print(f"Símbolo inicial: {simbolo_inicial}")

Terminales: ['or', 'and', 'not', 'true', 'false', '(', ')']
No-terminales: ['E', 'T', 'F']
Símbolo inicial: E


---
## Ejercicio 2: Tokenizador Básico

**Tarea:** Implementa un tokenizador para expresiones aritméticas.

In [2]:
import re

def tokenizar(expresion):
    """
    Tokeniza una expresión aritmética.
    Retorna: lista de tuplas (tipo, valor)
    """
    patrones = [
        ('NUMERO', r'\d+'),
        ('SUMA', r'\+'),
        ('RESTA', r'-'),
        ('MULT', r'\*'),
        ('DIV', r'/'),
        ('LPAREN', r'\('),
        ('RPAREN', r'\)'),
        ('ESPACIO', r'\s+'),
    ]

    # Combinar todos los patrones
    regex = '|'.join(f'(?P<{nombre}>{patron})' for nombre, patron in patrones)

    tokens = []
    for match in re.finditer(regex, expresion):
        tipo = match.lastgroup
        valor = match.group()
        if tipo != 'ESPACIO':  # Ignorar espacios
            tokens.append((tipo, valor))

    return tokens

# Pruebas
print(tokenizar("3 + 4 * 2"))
print(tokenizar("(10 - 5) / 2"))

[('NUMERO', '3'), ('SUMA', '+'), ('NUMERO', '4'), ('MULT', '*'), ('NUMERO', '2')]
[('LPAREN', '('), ('NUMERO', '10'), ('RESTA', '-'), ('NUMERO', '5'), ('RPAREN', ')'), ('DIV', '/'), ('NUMERO', '2')]


## Ejercicio 3: Clases para AST

**Tarea:** Define las clases para representar un AST.

In [3]:
class NodoAST:
    """Clase base para nodos del AST"""
    pass

class Numero(NodoAST):
    def __init__(self, valor):
        self.valor = int(valor)

    def __repr__(self):
        return f"Numero({self.valor})"

class OperacionBinaria(NodoAST):
    def __init__(self, izquierda, operador, derecha):
        self.izquierda = izquierda
        self.operador = operador
        self.derecha = derecha

    def __repr__(self):
        return f"OpBin({self.izquierda} {self.operador} {self.derecha})"

# Ejemplo de uso
ast = OperacionBinaria(
    Numero(3),
    '+',
    OperacionBinaria(Numero(4), '*', Numero(2))
)
print(f"AST para '3 + 4 * 2': {ast}")

AST para '3 + 4 * 2': OpBin(Numero(3) + OpBin(Numero(4) * Numero(2)))


---
## Ejercicio 4: Parser de Descenso Recursivo

Gramática sin recursión izquierda:
```
E -> T E'
E' -> + T E' | ε
T -> F T'
T' -> * F T' | ε
F -> ( E ) | numero
```

**Tarea:** Implementa el parser.

In [4]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0

    def token_actual(self):
        if self.pos < len(self.tokens):
            return self.tokens[self.pos]
        return None

    def consumir(self, tipo_esperado):
        token = self.token_actual()
        if token and token[0] == tipo_esperado:
            self.pos += 1
            return token
        raise SyntaxError(f"Se esperaba {tipo_esperado}, se encontró {token}")

    def parse_E(self):
        """E -> T E'"""
        izquierda = self.parse_T()
        return self.parse_E_prima(izquierda)

    def parse_E_prima(self, izquierda):
        """E' -> + T E' | ε"""
        token = self.token_actual()
        if token and token[0] == 'SUMA':
            self.consumir('SUMA')
            derecha = self.parse_T()
            nodo = OperacionBinaria(izquierda, '+', derecha)
            return self.parse_E_prima(nodo)
        elif token and token[0] == 'RESTA':
            self.consumir('RESTA')
            derecha = self.parse_T()
            nodo = OperacionBinaria(izquierda, '-', derecha)
            return self.parse_E_prima(nodo)
        return izquierda  # ε

    def parse_T(self):
        """T -> F T'"""
        izquierda = self.parse_F()
        return self.parse_T_prima(izquierda)

    def parse_T_prima(self, izquierda):
        """T' -> * F T' | ε"""
        token = self.token_actual()
        if token and token[0] == 'MULT':
            self.consumir('MULT')
            derecha = self.parse_F()
            nodo = OperacionBinaria(izquierda, '*', derecha)
            return self.parse_T_prima(nodo)
        elif token and token[0] == 'DIV':
            self.consumir('DIV')
            derecha = self.parse_F()
            nodo = OperacionBinaria(izquierda, '/', derecha)
            return self.parse_T_prima(nodo)
        return izquierda  # ε

    def parse_F(self):
        """F -> ( E ) | numero"""
        token = self.token_actual()
        if token[0] == 'LPAREN':
            self.consumir('LPAREN')
            nodo = self.parse_E()
            self.consumir('RPAREN')
            return nodo
        elif token[0] == 'NUMERO':
            self.consumir('NUMERO')
            return Numero(token[1])
        else:
            raise SyntaxError(f"Token inesperado: {token}")

    def parse(self):
        ast = self.parse_E()
        if self.pos != len(self.tokens):
            raise SyntaxError("Tokens adicionales después del final")
        return ast

# Prueba
expresion = "3 + 4 * 2"
tokens = tokenizar(expresion)
parser = Parser(tokens)
ast = parser.parse()
print(f"Expresión: {expresion}")
print(f"AST: {ast}")

Expresión: 3 + 4 * 2
AST: OpBin(Numero(3) + OpBin(Numero(4) * Numero(2)))




```
# Esto tiene formato de código
```

## Ejercicio 5: Evaluador de Expresiones

**Tarea:** Implementa un evaluador que calcule el resultado.

In [5]:
def evaluar(nodo):
    """Evalúa un AST y retorna el resultado."""
    if isinstance(nodo, Numero):
        return nodo.valor
    elif isinstance(nodo, OperacionBinaria):
        izq = evaluar(nodo.izquierda)
        der = evaluar(nodo.derecha)

        if nodo.operador == '+':
            return izq + der
        elif nodo.operador == '-':
            return izq - der
        elif nodo.operador == '*':
            return izq * der
        elif nodo.operador == '/':
            return izq / der

    raise ValueError(f"Nodo desconocido: {type(nodo)}")

# Pruebas
expresiones = [
    "3 + 4",
    "10 - 5 * 2",
    "(4 + 5) * 3",
    "100 / (2 + 3)"
]

for expr in expresiones:
    tokens = tokenizar(expr)
    parser = Parser(tokens)
    ast = parser.parse()
    resultado = evaluar(ast)
    print(f"{expr} = {resultado}")

3 + 4 = 7
10 - 5 * 2 = 0
(4 + 5) * 3 = 27
100 / (2 + 3) = 20.0


---
## Ejercicio 6: Calculadora Completa

**Tarea:** Integra todo en un sistema completo.

In [6]:
def calcular(expresion):
    """Función todo-en-uno: tokenizar -> parsear -> evaluar"""
    try:
        tokens = tokenizar(expresion)
        parser = Parser(tokens)
        ast = parser.parse()
        resultado = evaluar(ast)
        return resultado
    except Exception as e:
        return f"Error: {e}"

# Modo interactivo
print("Calculadora Aritmética")
print("Escribe 'salir' para terminar\n")

# Descomenta para modo interactivo:
# while True:
#     expresion = input(">>> ").strip()
#     if expresion.lower() == 'salir':
#         break
#     if expresion:
#         resultado = calcular(expresion)
#         print(f"  = {resultado}\n")

# Pruebas automáticas
pruebas = [
    "2 + 2",
    "10 * (5 + 3)",
    "100 / 4 - 5",
    "(8 - 3) * (4 + 2)"
]

for expr in pruebas:
    print(f"{expr} = {calcular(expr)}")

Calculadora Aritmética
Escribe 'salir' para terminar

2 + 2 = 4
10 * (5 + 3) = 80
100 / 4 - 5 = 20.0
(8 - 3) * (4 + 2) = 30


## Ejercicio 7: Detección de Ambigüedad

**Objetivo:** Identificar gramáticas ambiguas y entender sus problemas.

Gramática ambigua:
```
E -> E + E | E * E | ( E ) | id
```

**Tarea:** Demuestra que es ambigua generando dos árboles diferentes para: **`id + id * id`**


In [7]:
# Representa los árboles como listas de tuplas (nivel, nodo)

# Interpretación 1: (id + id) * id  [SUMA primero]
arbol_interpretacion_1 = """
        E
       /|\n      E * E
     /|\  |
    E+E  id
    | |
   id id
"""

# Interpretación 2: id + (id * id)  [MULTIPLICACIÓN primero]
arbol_interpretacion_2 = """
        E
       /|\n      E + E
      |  /|\n     id E*E
        | |
       id id
"""

print("="*70)
print("DEMOSTRACIÓN DE AMBIGÜEDAD: id + id * id")
print("="*70)

print("\nINTERPRETACIÓN 1: (id + id) * id")
print(arbol_interpretacion_1)
print("Resultado si id=2: (2 + 2) * 2 = 8")

print("\nINTERPRETACIÓN 2: id + (id * id)")
print(arbol_interpretacion_2)
print("Resultado si id=2: 2 + (2 * 2) = 6")

print("\n" + "="*70)
print("CONCLUSIÓN:")
print("="*70)
print("✓ La gramática es AMBIGUA porque la misma cadena tiene dos")
print("  árboles de derivación diferentes con significados distintos.")
print("✓ Esto es INACEPTABLE en lenguajes de programación.")

# PREGUNTA DE REFLEXIÓN:
respuesta = """
¿Cómo se resuelve esta ambigüedad?
1. Reescribir la gramática con niveles de precedencia (E, T, F)
2. Usar reglas de precedencia en el parser
3. Forzar asociatividad (izquierda o derecha)
"""
print("\n" + respuesta)

DEMOSTRACIÓN DE AMBIGÜEDAD: id + id * id

INTERPRETACIÓN 1: (id + id) * id

        E
       /|
      E * E
     /|\  |
    E+E  id
    | |
   id id

Resultado si id=2: (2 + 2) * 2 = 8

INTERPRETACIÓN 2: id + (id * id)

        E
       /|
      E + E
      |  /|
     id E*E
        | |
       id id

Resultado si id=2: 2 + (2 * 2) = 6

CONCLUSIÓN:
✓ La gramática es AMBIGUA porque la misma cadena tiene dos
  árboles de derivación diferentes con significados distintos.
✓ Esto es INACEPTABLE en lenguajes de programación.


¿Cómo se resuelve esta ambigüedad?
1. Reescribir la gramática con niveles de precedencia (E, T, F)
2. Usar reglas de precedencia en el parser
3. Forzar asociatividad (izquierda o derecha)



<>:7: SyntaxWarning: invalid escape sequence '\ '
<>:7: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-2855246149.py:7: SyntaxWarning: invalid escape sequence '\ '
  /|\  |


## Ejercicio 8: Árbol de Derivación vs AST

**Objetivo:** Diferenciar entre parse tree (derivación) y AST (abstracto).

Para la expresión: **`3 + 4 * 5`**


In [8]:
# Árbol de Derivación (Parse Tree) - muestra TODA la derivación
parse_tree = """
Árbol de Derivación (Parse Tree):
Muestra CÓMO se aplicó la gramática

            E
           /|\n          E + T
          |  /|\n          T T * F
          | |   |
          F F   5
          | |
          3 4

Características:
- Incluye TODOS los no-terminales (E, T, F)
- Muestra la estructura gramatical completa
- Útil para verificar que la derivación es correcta
"""

# Árbol de Sintaxis Abstracto (AST) - solo lo esencial
ast_tree = """
Árbol de Sintaxis Abstracto (AST):
Captura solo la ESTRUCTURA ESENCIAL

        +
       / \n      3   *
         / \n        4   5

Características:
- OMITE no-terminales intermedios (E, T, F)
- Solo operadores y operandos
- Más limpio y fácil de procesar
- Usado por fases posteriores del compilador
"""

print("="*70)
print("COMPARACIÓN: Parse Tree vs AST")
print("="*70)
print(parse_tree)
print("\n" + "="*70)
print(ast_tree)

# Implementación de clases para AST
class NodoAST:
    pass

class Numero(NodoAST):
    def __init__(self, valor):
        self.valor = valor
    def __repr__(self):
        return str(self.valor)

class OpBinaria(NodoAST):
    def __init__(self, izq, op, der):
        self.izq = izq
        self.op = op
        self.der = der
    def __repr__(self):
        return f"({self.izq} {self.op} {self.der})"

# Construye el AST para 3 + 4 * 5
ast = OpBinaria(
    Numero(3),
    '+',
    OpBinaria(Numero(4), '*', Numero(5))
)

print("\n" + "="*70)
print("AST como objeto Python:")
print(f"  {ast}")
print("\n✓ El AST es la representación preferida para análisis semántico")
print("  y generación de código.")

COMPARACIÓN: Parse Tree vs AST

Árbol de Derivación (Parse Tree):
Muestra CÓMO se aplicó la gramática

            E
           /|
          E + T
          |  /|
          T T * F
          | |   |
          F F   5
          | |
          3 4

Características:
- Incluye TODOS los no-terminales (E, T, F)
- Muestra la estructura gramatical completa
- Útil para verificar que la derivación es correcta



Árbol de Sintaxis Abstracto (AST):
Captura solo la ESTRUCTURA ESENCIAL

        +
       / 
      3   *
         / 
        4   5

Características:
- OMITE no-terminales intermedios (E, T, F)
- Solo operadores y operandos
- Más limpio y fácil de procesar
- Usado por fases posteriores del compilador


AST como objeto Python:
  (3 + (4 * 5))

✓ El AST es la representación preferida para análisis semántico
  y generación de código.


## Ejercicio 9: Transformación - Eliminación de Recursión Izquierda

**Objetivo:** Aprender a transformar gramáticas para análisis descendente.

**Problema:** Los parsers descendentes no pueden manejar recursión izquierda.

Gramática CON recursión izquierda:
```
E -> E + T | T
T -> T * F | F
F -> ( E ) | id
```


In [9]:
# Algoritmo para eliminar recursión izquierda:
# Si tenemos: A -> A α | β
# Transformar a: A -> β A'
#                A' -> α A' | ε

print("="*70)
print("ELIMINACIÓN DE RECURSIÓN IZQUIERDA")
print("="*70)

print("\nGRAMÁTICA ORIGINAL (con recursión izquierda):")
gramatica_original = {
    'E': ['E + T', 'T'],
    'T': ['T * F', 'F'],
    'F': ['( E )', 'id']
}

for nt, prods in gramatica_original.items():
    for prod in prods:
        print(f"  {nt} -> {prod}")

print("\n⚠️  PROBLEMA: E -> E + T y T -> T * F causan recursión infinita")
print("    en parsers descendentes.\n")

# COMPLETA LA GRAMÁTICA TRANSFORMADA:
print("GRAMÁTICA TRANSFORMADA (sin recursión izquierda):")
gramatica_transformada = {
    'E': [],      # COMPLETA: ['T E\'']
    "E'": [],     # COMPLETA: ['+ T E\'', 'ε']
    'T': [],      # COMPLETA: ['F T\'']
    "T'": [],     # COMPLETA: ['* F T\'', 'ε']
    'F': ['( E )', 'id']
}

# SOLUCIÓN:
solucion = {
    'E': ['T E\''],
    "E'": ['+ T E\'', 'ε'],
    'T': ['F T\''],
    "T'": ['* F T\'', 'ε'],
    'F': ['( E )', 'id']
}

for nt, prods in solucion.items():
    for prod in prods:
        print(f"  {nt} -> {prod}")

print("\n✓ Ahora la gramática NO tiene recursión izquierda")
print("✓ Puede ser usada por un parser de descenso recursivo")

ELIMINACIÓN DE RECURSIÓN IZQUIERDA

GRAMÁTICA ORIGINAL (con recursión izquierda):
  E -> E + T
  E -> T
  T -> T * F
  T -> F
  F -> ( E )
  F -> id

⚠️  PROBLEMA: E -> E + T y T -> T * F causan recursión infinita
    en parsers descendentes.

GRAMÁTICA TRANSFORMADA (sin recursión izquierda):
  E -> T E'
  E' -> + T E'
  E' -> ε
  T -> F T'
  T' -> * F T'
  T' -> ε
  F -> ( E )
  F -> id

✓ Ahora la gramática NO tiene recursión izquierda
✓ Puede ser usada por un parser de descenso recursivo



## Ejercicio 10: Implementación de Tokenizador Robusto

**Objetivo:** Crear un tokenizador completo para expresiones aritméticas.


In [10]:
import re

def tokenizar(expresion):
    """
    Tokeniza una expresión aritmética.
    Retorna: lista de tuplas (tipo, valor, posicion)
    """
    # Define los patrones de tokens (orden importa!)
    patrones = [
        ('NUMERO',   r'\d+(\.\d+)?'),  # Enteros y decimales
        ('ID',       r'[a-zA-Z_][a-zA-Z0-9_]*'),  # Identificadores
        ('SUMA',     r'\+'),
        ('RESTA',    r'-'),
        ('MULT',     r'\*'),
        ('DIV',      r'/'),
        ('POT',      r'\^'),
        ('LPAREN',   r'\('),
        ('RPAREN',   r'\)'),
        ('ESPACIO',  r'\s+'),
        ('ERROR',    r'.'),  # Cualquier carácter no reconocido
    ]

    # Combinar todos los patrones en una sola regex
    regex_completa = '|'.join(f'(?P<{nombre}>{patron})' for nombre, patron in patrones)

    tokens = []
    for match in re.finditer(regex_completa, expresion):
        tipo = match.lastgroup
        valor = match.group()
        posicion = match.start()

        if tipo == 'ESPACIO':
            continue  # Ignorar espacios
        elif tipo == 'ERROR':
            raise SyntaxError(f"Carácter inválido '{valor}' en posición {posicion}")

        tokens.append((tipo, valor, posicion))

    return tokens

# Pruebas
print("="*70)
print("TOKENIZADOR DE EXPRESIONES ARITMÉTICAS")
print("="*70)

expresiones_prueba = [
    "3 + 4",
    "10 * (2 + 5)",
    "x + y * 2",
    "3.14 * radio ^ 2",
]

for expr in expresiones_prueba:
    print(f"\nExpresión: {expr}")
    tokens = tokenizar(expr)
    print("Tokens:")
    for tipo, valor, pos in tokens:
        print(f"  {tipo:10} '{valor:5}' (pos {pos})")

print("\n✓ El tokenizador es el primer paso del análisis sintáctico")

TOKENIZADOR DE EXPRESIONES ARITMÉTICAS

Expresión: 3 + 4
Tokens:
  NUMERO     '3    ' (pos 0)
  SUMA       '+    ' (pos 2)
  NUMERO     '4    ' (pos 4)

Expresión: 10 * (2 + 5)
Tokens:
  NUMERO     '10   ' (pos 0)
  MULT       '*    ' (pos 3)
  LPAREN     '(    ' (pos 5)
  NUMERO     '2    ' (pos 6)
  SUMA       '+    ' (pos 8)
  NUMERO     '5    ' (pos 10)
  RPAREN     ')    ' (pos 11)

Expresión: x + y * 2
Tokens:
  ID         'x    ' (pos 0)
  SUMA       '+    ' (pos 2)
  ID         'y    ' (pos 4)
  MULT       '*    ' (pos 6)
  NUMERO     '2    ' (pos 8)

Expresión: 3.14 * radio ^ 2
Tokens:
  NUMERO     '3.14 ' (pos 0)
  MULT       '*    ' (pos 5)
  ID         'radio' (pos 7)
  POT        '^    ' (pos 13)
  NUMERO     '2    ' (pos 15)

✓ El tokenizador es el primer paso del análisis sintáctico


## Ejercicio 11: Parser de Descenso Recursivo Completo

**Objetivo:** Implementar un parser descendente con precedencia de operadores.

Gramática (sin recursión izquierda):
```
E -> T E'
E' -> + T E' | - T E' | ε
T -> F T'
T' -> * F T' | / F T' | ε
F -> ( E ) | numero | id
```


In [11]:
# Clases para el AST
class NodoAST:
    pass

class Numero(NodoAST):
    def __init__(self, valor):
        self.valor = float(valor) if '.' in str(valor) else int(valor)

    def __repr__(self):
        return f"{self.valor}"

    def evaluar(self, variables=None):
        return self.valor

class Variable(NodoAST):
    def __init__(self, nombre):
        self.nombre = nombre

    def __repr__(self):
        return self.nombre

    def evaluar(self, variables=None):
        if variables and self.nombre in variables:
            return variables[self.nombre]
        raise NameError(f"Variable '{self.nombre}' no definida")

class OperacionBinaria(NodoAST):
    def __init__(self, izquierda, operador, derecha):
        self.izquierda = izquierda
        self.operador = operador
        self.derecha = derecha

    def __repr__(self):
        return f"({self.izquierda} {self.operador} {self.derecha})"

    def evaluar(self, variables=None):
        izq = self.izquierda.evaluar(variables)
        der = self.derecha.evaluar(variables)

        if self.operador == '+':
            return izq + der
        elif self.operador == '-':
            return izq - der
        elif self.operador == '*':
            return izq * der
        elif self.operador == '/':
            if der == 0:
                raise ZeroDivisionError("División por cero")
            return izq / der
        elif self.operador == '^':
            return izq ** der

# Parser de Descenso Recursivo
class ParserDescensoRecursivo:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0

    def token_actual(self):
        """Retorna el token actual sin consumirlo."""
        if self.pos < len(self.tokens):
            return self.tokens[self.pos]
        return None

    def consumir(self, tipo_esperado):
        """Consume un token del tipo esperado."""
        token = self.token_actual()
        if not token:
            raise SyntaxError(f"Se esperaba {tipo_esperado}, pero se encontró el final")

        tipo, valor, pos = token
        if tipo != tipo_esperado:
            raise SyntaxError(
                f"Se esperaba {tipo_esperado}, pero se encontró {tipo} en posición {pos}"
            )

        self.pos += 1
        return valor

    def parse_E(self):
        """E -> T E'"""
        nodo = self.parse_T()
        return self.parse_E_prima(nodo)

    def parse_E_prima(self, izquierda):
        """E' -> + T E' | - T E' | ε"""
        token = self.token_actual()
        if token and token[0] in ['SUMA', 'RESTA']:
            operador = '+' if token[0] == 'SUMA' else '-'
            self.consumir(token[0])
            derecha = self.parse_T()
            nodo = OperacionBinaria(izquierda, operador, derecha)
            return self.parse_E_prima(nodo)  # Asociatividad izquierda
        return izquierda  # ε (epsilon)

    def parse_T(self):
        """T -> F T'"""
        nodo = self.parse_F()
        return self.parse_T_prima(nodo)

    def parse_T_prima(self, izquierda):
        """T' -> * F T' | / F T' | ε"""
        token = self.token_actual()
        if token and token[0] in ['MULT', 'DIV']:
            operador = '*' if token[0] == 'MULT' else '/'
            self.consumir(token[0])
            derecha = self.parse_F()
            nodo = OperacionBinaria(izquierda, operador, derecha)
            return self.parse_T_prima(nodo)  # Asociatividad izquierda
        return izquierda  # ε (epsilon)

    def parse_F(self):
        """F -> ( E ) | numero | id"""
        token = self.token_actual()

        if not token:
            raise SyntaxError("Se esperaba un factor pero se encontró el final")

        tipo, valor, pos = token

        if tipo == 'LPAREN':
            self.consumir('LPAREN')
            nodo = self.parse_E()
            self.consumir('RPAREN')
            return nodo
        elif tipo == 'NUMERO':
            self.consumir('NUMERO')
            return Numero(valor)
        elif tipo == 'ID':
            self.consumir('ID')
            return Variable(valor)
        else:
            raise SyntaxError(f"Token inesperado: {tipo} en posición {pos}")

    def parse(self):
        """Punto de entrada del parser."""
        ast = self.parse_E()
        if self.pos != len(self.tokens):
            tipo, valor, pos = self.tokens[self.pos]
            raise SyntaxError(f"Tokens adicionales después del final: {tipo} en posición {pos}")
        return ast

# Pruebas
print("="*70)
print("PARSER DE DESCENSO RECURSIVO")
print("="*70)

pruebas = [
    ("3 + 4", {}),
    ("10 * 2 + 5", {}),
    ("(4 + 5) * 3", {}),
    ("x + y * 2", {'x': 5, 'y': 3}),
]

for expr, vars in pruebas:
    print(f"\nExpresión: {expr}")
    if vars:
        print(f"Variables: {vars}")

    try:
        tokens = tokenizar(expr)
        parser = ParserDescensoRecursivo(tokens)
        ast = parser.parse()
        resultado = ast.evaluar(vars)

        print(f"AST: {ast}")
        print(f"Resultado: {resultado}")
        print("✓ Parsing exitoso")
    except Exception as e:
        print(f"✗ Error: {e}")

print("\n" + "="*70)
print("✓ El parser descendente construye el AST de arriba hacia abajo")

PARSER DE DESCENSO RECURSIVO

Expresión: 3 + 4
AST: (3 + 4)
Resultado: 7
✓ Parsing exitoso

Expresión: 10 * 2 + 5
AST: ((10 * 2) + 5)
Resultado: 25
✓ Parsing exitoso

Expresión: (4 + 5) * 3
AST: ((4 + 5) * 3)
Resultado: 27
✓ Parsing exitoso

Expresión: x + y * 2
Variables: {'x': 5, 'y': 3}
AST: (x + (y * 2))
Resultado: 11
✓ Parsing exitoso

✓ El parser descendente construye el AST de arriba hacia abajo


## Ejercicio 12: Tabla de Análisis LL(1)

**Objetivo:** Construir una tabla LL(1) y usarla para parsing.

Gramática:
```
E -> T E'
E' -> + T E' | ε
T -> F T'
T' -> * F T' | ε
F -> ( E ) | id
```


In [12]:
# Tabla LL(1): tabla[no_terminal][terminal] = producción
tabla_ll1 = {
    'E': {
        'id': "T E'",
        '(': "T E'",
    },
    "E'": {
        '+': "+ T E'",
        ')': "ε",
        '': "ε",
    },
    'T': {
        'id': "F T'",
        '(': "F T'",
    },
    "T'": {
        '+': "ε",
        '*': "* F T'",
        ')': "ε",
        '': "ε",
    },
    'F': {
        'id': "id",
        '(': "( E )",
    }
}

def mostrar_tabla_ll1(tabla):
    """Imprime la tabla LL(1) de forma legible."""
    terminales = set()
    for producciones in tabla.values():
        terminales.update(producciones.keys())
    terminales = sorted(terminales, key=lambda x: (x != 'id', x != '(', x))

    print(f"{'':8}", end="")
    for t in terminales:
        print(f"{t:12}", end="")
    print()
    print("-" * (8 + 12 * len(terminales)))

    for nt in tabla:
        print(f"{nt:8}", end="")
        for t in terminales:
            prod = tabla[nt].get(t, '')
            print(f"{prod:12}", end="")
        print()

print("="*70)
print("TABLA DE ANÁLISIS LL(1)")
print("="*70)
print()
mostrar_tabla_ll1(tabla_ll1)

# Parser dirigido por tabla
def parser_ll1_tabla(entrada, tabla, simbolo_inicial='E'):
    """
    Parser LL(1) dirigido por tabla.
    """
    # Preparar entrada
    tokens_entrada = entrada + ['']
    pila = ['', simbolo_inicial]
    i = 0  # Índice en la entrada

    print("\n" + "="*90)
    print(f"{'Pila':<30} {'Entrada':<25} {'Acción'}")
    print("-" * 90)

    while len(pila) > 0:
        tope = pila[-1]
        actual = tokens_entrada[i]

        # Mostrar estado
        pila_str = ' '.join(reversed(pila))
        entrada_str = ' '.join(tokens_entrada[i:])

        if tope == ' and actual == ':
            print(f"{pila_str:<30} {entrada_str:<25} ACEPTAR")
            return True

        elif tope == actual:
            # Match: consumir terminal
            print(f"{pila_str:<30} {entrada_str:<25} Match '{tope}'")
            pila.pop()
            i += 1

        elif tope in tabla and actual in tabla[tope]:
            # Expandir no-terminal
            produccion = tabla[tope][actual]
            print(f"{pila_str:<30} {entrada_str:<25} {tope} -> {produccion}")
            pila.pop()

            if produccion != 'ε':
                # Agregar producción a la pila (en orden inverso)
                simbolos = produccion.split()
                for simbolo in reversed(simbolos):
                    pila.append(simbolo)

        else:
            print(f"{pila_str:<30} {entrada_str:<25} ERROR")
            return False

    return False

# Prueba
print("\n" + "="*70)
print("TRAZA DE ANÁLISIS PARA: id + id * id")
print("="*70)
resultado = parser_ll1_tabla(['id', '+', 'id', '*', 'id'], tabla_ll1)
print("\n✓ El parser LL(1) usa la tabla para decidir qué producción aplicar")

TABLA DE ANÁLISIS LL(1)

        id          (                       )           *           +           
--------------------------------------------------------------------------------
E       T E'        T E'                                                        
E'                              ε           ε                       + T E'      
T       F T'        F T'                                                        
T'                              ε           ε           * F T'      ε           
F       id          ( E )                                                       

TRAZA DE ANÁLISIS PARA: id + id * id

Pila                           Entrada                   Acción
------------------------------------------------------------------------------------------
E                              id + id * id              E -> T E'
T E'                           id + id * id              T -> F T'
F T' E'                        id + id * id              F -> id
id T' E'       

## Ejercicio 13: Conceptos de Análisis Ascendente

**Objetivo:** Comprender las operaciones shift y reduce.

**Análisis Ascendente:** Construye el árbol desde las hojas hacia la raíz.

**Operaciones principales:**
- **SHIFT:** Desplazar (push) un token de la entrada a la pila
- **REDUCE:** Reducir (pop) símbolos de la pila usando una producción
- **ACCEPT:** Aceptar la cadena
- **ERROR:** Rechazar la cadena


In [13]:
print("="*70)
print("ANÁLISIS ASCENDENTE: SHIFT-REDUCE")
print("="*70)

print("""
CONCEPTO FUNDAMENTAL:
====================
El parser ascendente lee tokens de izquierda a derecha y:
1. DESPLAZA (shift) tokens a una pila
2. Cuando reconoce el lado derecho de una producción en la pila,
   lo REDUCE al no-terminal correspondiente
3. Continúa hasta reducir todo al símbolo inicial

Ejemplo: Parsear "id + id" con gramática E -> E + E | id

Paso  Pila           Entrada      Acción
----  -------------  -----------  -------------------------
1     $              id + id $    shift
2     $ id           + id $       reduce (id -> E)
3     $ E            + id $       shift
4     $ E +          id $         shift
5     $ E + id       $            reduce (id -> E)
6     $ E + E        $            reduce (E + E -> E)
7     $ E            $            ACCEPT

VENTAJAS del análisis ascendente:
✓ Maneja gramáticas con recursión izquierda
✓ Más potente que el análisis descendente
✓ Usado por la mayoría de generadores (YACC, Bison)

DESVENTAJAS:
✗ Más complejo de entender e implementar
✗ Requiere tablas de análisis complejas
✗ Más difícil de depurar manualmente
""")

print("="*70)
print("PREGUNTA: ¿Cuándo hacer SHIFT y cuándo REDUCE?")
print("="*70)
print("""
Esta es la pregunta clave del análisis ascendente.
La respuesta depende del tipo de parser:

- Parser LR(0): Usa solo la pila (estados)
- Parser SLR: Usa pila + 1 token de lookahead
- Parser LALR: Versión optimizada de LR(1)
- Parser LR(1): Usa pila + k tokens de lookahead

Todos usan TABLAS DE ANÁLISIS preconstruidas.
""")

ANÁLISIS ASCENDENTE: SHIFT-REDUCE

CONCEPTO FUNDAMENTAL:
El parser ascendente lee tokens de izquierda a derecha y:
1. DESPLAZA (shift) tokens a una pila
2. Cuando reconoce el lado derecho de una producción en la pila,
   lo REDUCE al no-terminal correspondiente
3. Continúa hasta reducir todo al símbolo inicial

Ejemplo: Parsear "id + id" con gramática E -> E + E | id

Paso  Pila           Entrada      Acción
----  -------------  -----------  -------------------------
1     $              id + id $    shift
2     $ id           + id $       reduce (id -> E)
3     $ E            + id $       shift
4     $ E +          id $         shift
5     $ E + id       $            reduce (id -> E)
6     $ E + E        $            reduce (E + E -> E)
7     $ E            $            ACCEPT

VENTAJAS del análisis ascendente:
✓ Maneja gramáticas con recursión izquierda
✓ Más potente que el análisis descendente
✓ Usado por la mayoría de generadores (YACC, Bison)

DESVENTAJAS:
✗ Más complejo de entend

## Ejercicio 14: Identificación de Handles

**Objetivo:** Aprender a identificar el "handle" (mango) para reducir.

**Handle:** Subcadena que coincide con el lado derecho de una producción y cuya reducción nos acerca al símbolo inicial.


In [14]:
print("="*70)
print("IDENTIFICACIÓN DE HANDLES")
print("="*70)

gramatica = """
Gramática:
  E -> E + T
  E -> T
  T -> T * F
  T -> F
  F -> ( E )
  F -> id
"""
print(gramatica)

# Ejercicio: identificar handles
ejemplos = [
    {
        'forma_sentencial': 'E + T * id',
        'handle': 'id',
        'produccion': 'F -> id',
        'explicacion': 'id se puede reducir a F'
    },
    {
        'forma_sentencial': 'E + T * F',
        'handle': 'T * F',
        'produccion': 'T -> T * F',
        'explicacion': 'T * F se reduce a T'
    },
    {
        'forma_sentencial': 'E + T',
        'handle': 'E + T',
        'produccion': 'E -> E + T',
        'explicacion': 'E + T se reduce a E'
    },
]

print("\nEjemplos de identificación de handles:")
print("-" * 70)

for i, ej in enumerate(ejemplos, 1):
    print(f"\nEjemplo {i}:")
    print(f"  Forma sentencial: {ej['forma_sentencial']}")
    print(f"  Handle: {ej['handle']}")
    print(f"  Producción: {ej['produccion']}")
    print(f"  Explicación: {ej['explicacion']}")

print("\n" + "="*70)
print("REGLA GENERAL: El handle es la subcadena más a la derecha")
print("que se puede reducir en una derivación rightmost en reversa.")
print("="*70)


IDENTIFICACIÓN DE HANDLES

Gramática:
  E -> E + T
  E -> T
  T -> T * F
  T -> F
  F -> ( E )
  F -> id


Ejemplos de identificación de handles:
----------------------------------------------------------------------

Ejemplo 1:
  Forma sentencial: E + T * id
  Handle: id
  Producción: F -> id
  Explicación: id se puede reducir a F

Ejemplo 2:
  Forma sentencial: E + T * F
  Handle: T * F
  Producción: T -> T * F
  Explicación: T * F se reduce a T

Ejemplo 3:
  Forma sentencial: E + T
  Handle: E + T
  Producción: E -> E + T
  Explicación: E + T se reduce a E

REGLA GENERAL: El handle es la subcadena más a la derecha
que se puede reducir en una derivación rightmost en reversa.


### Próximos Pasos
1. Estudiar análisis semántico
2. Aprender sobre generación de código
3. Implementar optimizaciones
4. Crear un lenguaje completo
